In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
data = pd.read_csv("final_with_time.csv")

In [4]:
data = data[data['milliseconds'] != 0]
data = data[data['position'] != 0]
data.columns[7:20]

Index(['weather_warm', 'weather_cold', 'weather_dry', 'weather_wet',
       'weather_cloudy', 'age', 'Win', 'podium', 'circuitId_2', 'circuitId_3',
       'circuitId_4', 'circuitId_5', 'circuitId_6'],
      dtype='object')

In [183]:
x = data.drop(["position","podium",'Win', "milliseconds"],axis=1)
y = data[["raceId","milliseconds","position"]]

In [184]:
reg = LinearRegression().fit(x, y.milliseconds)
predictions = np.argsort(reg.predict(x))
accuracy = np.sum(predictions == y.position)/len(y)

metrics(y.position, predictions)

[0.7893623451193714,
 0.6488365064974313,
 0.6947718343910547,
 0.3466304019341191]

In [173]:
# drop rows of y if milliseconds == zero
x = x[y['position'] != 0]
x = x[y['milliseconds'] != 0]
y = y[y['position'] != 0]
y = y[y['milliseconds'] != 0]
gY = []

for group in data.groupby("raceId"):
    group[1].milliseconds = group[1].milliseconds - group[1].milliseconds.min()
    x = group[1].drop(["position","podium",'Win', "milliseconds"],axis=1)
    y = group[1][["raceId","milliseconds","position"]]
    # run a linear regression
    reg = LinearRegression().fit(x, y.milliseconds)
    predictions = np.argsort(reg.predict(x))
    accuracy = np.sum(predictions == y.position)/len(y)

    gY.append(metrics(y.position, predictions))

In [176]:
df = pd.DataFrame(gY)
df.columns = ["top3", "top5", "top10", "top20"]

In [177]:
df

,top3,top5,top10,top20
0,0.500000,0.357143,0.357143,0.428571
1,0.687500,0.687500,0.437500,0.500000
2,0.800000,0.533333,0.466667,0.266667
3,0.823529,0.764706,0.411765,0.294118
4,0.615385,0.461538,0.307692,0.538462
...,...,...,...,...
380,0.000000,0.000000,0.500000,0.500000
381,0.000000,0.000000,0.000000,1.000000
382,0.000000,0.000000,0.000000,1.000000
383,0.000000,0.000000,0.000000,1.000000


In [181]:
df.median(axis=0)

top3     0.615385
top5     0.562500
top10    0.600000
top20    0.666667
dtype: float64

In [179]:
df.mean(axis=0)

top3     0.514451
top5     0.505532
top10    0.611087
top20    0.652499
dtype: float64

In [182]:
df.mode(axis=0)

,top3,top5,top10,top20
0,0.0,0.0,1.0,1.0


# Hamza metrics

In [154]:
def metric_top3(y_true, y_pred):
    y_true = list(y_true)
    top3_true = []
    top3_pred = []
    for i in range(len(y_pred)):
        if y_true[i] <= 3:
            top3_true.append(1)
        else:
            top3_true.append(0)
        if y_pred[i] <= 3:
            top3_pred.append(1)
        else:
            top3_pred.append(0)
    
    return top3_true, top3_pred, accuracy_score(top3_true, top3_pred)

In [155]:
def metric_top5(y_true, y_pred):
    y_true = list(y_true)
    top_true = []
    top_pred = []
    for i in range(len(y_pred)):
        if y_true[i] <= 5:
            top_true.append(1)
        else:
            top_true.append(0)
        if y_pred[i] <= 5:
            top_pred.append(1)
        else:
            top_pred.append(0)
    
    return top_true, top_pred, accuracy_score(top_true, top_pred)

In [156]:
def metric_top6_10(y_true, y_pred):
    y_true = list(y_true)
    top_true = []
    top_pred = []
    for i in range(len(y_pred)):
        if y_true[i] <= 10 and y_true[i] >= 6:
            top_true.append(1)
        else:
            top_true.append(0)
        if y_pred[i] <= 10 and y_pred[i] >= 6:
            top_pred.append(1)
        else:
            top_pred.append(0)
    
    return top_true, top_pred, accuracy_score(top_true, top_pred)

In [157]:
def metric_top10_20(y_true, y_pred):
    y_true = list(y_true)
    top_true = []
    top_pred = []
    for i in range(len(y_pred)):
        if y_true[i] > 10:
            top_true.append(1)
        else:
            top_true.append(0)
        if y_pred[i] > 10:
            top_pred.append(1)
        else:
            top_pred.append(0)
    
    return top_true, top_pred, accuracy_score(top_true, top_pred)

In [170]:
def metrics(y_true, y_pred):
    y_true = list(y_true)
    top3_true = []
    top5_true = []
    top6_10_true = []
    top10_20_true = []
    top3_pred = []
    top5_pred = []
    top6_10_pred = []
    top10_20_pred = []
    for i in range(len(y_pred)):

        if y_true[i] <= 3:
            top3_true.append(1)
        else:
            top3_true.append(0)

        if y_true[i] <= 5:
            top5_true.append(1)
        else:
            top5_true.append(0)

        if y_true[i] <= 10 and y_true[i] >= 6:
            top6_10_true.append(1)
        else:
            top6_10_true.append(0)

        if y_true[i] > 10:
            top10_20_true.append(1)
        else:
            top10_20_true.append(0) 

            
        if y_pred[i] <= 3:
            top3_pred.append(1)
        else:
            top3_pred.append(0)

        if y_pred[i] <= 5:
            top5_pred.append(1)
        else:
            top5_pred.append(0)

        if y_pred[i] <= 10 and y_pred[i] >= 6:
            top6_10_pred.append(1)
        else:
            top6_10_pred.append(0)

        if y_pred[i] > 10:
            top10_20_pred.append(1)
        else:
            top10_20_pred.append(0)
            
    return [accuracy_score(top3_true, top3_pred), accuracy_score(top5_true, top5_pred), accuracy_score(top6_10_true, top6_10_pred), accuracy_score(top10_20_true, top10_20_pred)]